In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from IPython.display import Image
import glob
import struct
import open3d as o3d
from tqdm import tqdm
import pandas as pd

from segmentations.segmentation import segmentation_img, human_labeling
from argparse import Namespace
from PCDprocessing.lidar_camera_project import RGB_mapping2pcd, label_coloring2pcd
import argparse
import json
import os

In [3]:
## read json file
def load_config_as_args(json_path):
    """Load configuration from a JSON file and return it as an argparse.Namespace object."""
    if not os.path.exists(json_path):
        raise FileNotFoundError(f"Configuration file not found: {json_path}")
    with open(json_path, 'r') as file:
        config = json.load(file)

    # Convert dictionary to Namespace
    args = Namespace(**config)
    # args = parser.parse_args()
    return args
args = load_config_as_args("config.json")

point_files = sorted(glob.glob(args.pcd_path))
label_files = sorted(glob.glob(args.lbl_path))
image2_files = sorted(glob.glob(args.img2_path))
image3_files = sorted(glob.glob(args.img3_path))

## Calibration matrix composition
with open(args.calib_path,'r') as f:
    calib = f.readlines()

P2 = np.array([float(x) for x in calib[2].strip('\n').split(' ')[1:]]).reshape(3, 4)
P3 = np.array([float(x) for x in calib[3].strip('\n').split(' ')[1:]]).reshape(3, 4)
Tr = np.array([float(x) for x in calib[4].strip('\n').split(' ')[1:]]).reshape(3, 4)
Tr = np.insert(Tr, 3, values=[0, 0, 0, 1], axis=0)

In [4]:
file_index = 9
Person_label = 30
img = image2_files[file_index]
binary = point_files[file_index]
label = label_files[file_index]

origin_img = cv2.imread(img)
origin_img = cv2.cvtColor(origin_img, cv2.COLOR_BGR2RGB)
Person_label = 30
lbls = np.fromfile(label, dtype=np.uint32) & 0xFFFF

num2 = 0
for nums in lbls:
    if nums==Person_label:
        num2+=1
print("",num2)

# read raw data from binary
scan = np.fromfile(binary, dtype=np.float32).reshape((-1, 4))
points = scan[:, 0:3]  # lidar xyz (front, left, up)
points_idx = {}
for idxs in range(len(points)):
    points_idx[idxs] = points[idxs]

# Apply projection matrix
velo = np.insert(points, 3, 1, axis=1).T
cam = P2.dot(Tr.dot(velo))
cam[:2] /= cam[2, :]

# Preparation on image
plt.figure(figsize=(12, 5), dpi=96, tight_layout=True)
label_color = (255,1,1)
png = origin_img
IMG_H, IMG_W, _ = png.shape
# plt.axis([0, IMG_W, IMG_H, 0])/

# Filter point out of canvas
u, v, z = cam
u_out = np.logical_or(u < 0, u > IMG_W)
v_out = np.logical_or(v < 0, v > IMG_H)
outlier = np.logical_or(u_out, v_out)

remain_pts = []
for idxss, bools in enumerate(outlier):
    if not bools:
        if lbls[idxss] == Person_label:
            remain_pts.append(points[idxss])


 41


<Figure size 1152x480 with 0 Axes>

In [53]:
remain_pts = []
for idxss, bools in enumerate(outlier):
    if not bools:
        if lbls[idxss] == Person_label:
            remain_pts.append(points[idxss])

len(remain_pts)

227

# Test Field

In [ ]:
# file_index = 9
print(len(image2_files))
lbl_points_in_sight = []
for file_index in tqdm(range(len(image2_files))):
    Person_label = 30
    img = image2_files[file_index]
    binary = point_files[file_index]
    label = label_files[file_index]

    origin_img = cv2.imread(img)
    origin_img = cv2.cvtColor(origin_img, cv2.COLOR_BGR2RGB)
    Person_label = 30
    lbls = np.fromfile(label, dtype=np.uint32) & 0xFFFF

    # num2 = 0
    # for nums in lbls:
    #     if nums==Person_label:
    #         num2+=1
    # print("",num2)

    # read raw data from binary
    scan = np.fromfile(binary, dtype=np.float32).reshape((-1, 4))
    points = scan[:, 0:3]  # lidar xyz (front, left, up)
    points_idx = {}
    for idxs in range(len(points)):
        points_idx[idxs] = points[idxs]

    # Apply projection matrix
    velo = np.insert(points, 3, 1, axis=1).T
    cam = P2.dot(Tr.dot(velo))
    cam[:2] /= cam[2, :]

    # Preparation on image
    plt.figure(figsize=(12, 5), dpi=96, tight_layout=True)
    label_color = (255,1,1)
    png = origin_img
    IMG_H, IMG_W, _ = png.shape
    # plt.axis([0, IMG_W, IMG_H, 0])/

    # Filter point out of canvas
    u, v, z = cam
    u_out = np.logical_or(u < 0, u > IMG_W)
    v_out = np.logical_or(v < 0, v > IMG_H)
    z_out = np.logical_or(z < 0, z > 500000)
    outlier = np.logical_or(np.logical_or(u_out, v_out), z_out)

    remain_pts = []
    for idxss, bools in enumerate(outlier):
        if not bools:
            if lbls[idxss] == Person_label:
                remain_pts.append(points[idxss])
    
    lbl_points_in_sight.append(len(remain_pts))

In [6]:
datas= {'FoV_pts':lbl_points_in_sight}
df = pd.DataFrame(datas)
df.to_excel('./../data/outputs/PCD_denoise_SeKITTI_seq08_FoV.xlsx')